# Aufgabe 4:

In [2]:
#Import stuff & things
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Hilfsfunktionen

In [3]:
# Convenient function to output test items
def show_test_item(clf, i, text=True, classify=True):
    a = 'actual: ' + str(test.iloc[i].recommended)
    p = ''
    if classify:
        p = ' / predicted: ' + str(clf.predict(vectorizer.transform([test.iloc[i].content_text]))[0])
    print('[' + str(i) + '] ' +a + p)
    if text:
        print('\n')
        print(test.iloc[i].content_text)

# Dokumente laden

In [26]:
#Read Datasets for all Areas
dir = '/Users/juliusstahnke/Documents/HTW/Analytics/Analytics/Aufgabe4'

games = pd.read_json(dir + '/games/games.json', orient='records', encoding='utf8')
music = pd.read_json(dir + '/music/music.json', orient='records', encoding='utf8')
apps = pd.read_json(dir + '/apps/apps.json', orient='records', encoding='utf8')

# Aufbereiten Tabellen

# Aufspalten in Train und Test

In [27]:
#Set Randomn State
random_state = 100

#Set Traindata for games
train_games_positiv = games[games.recommended==True].sample(frac=0.8, random_state=random_state)
train_games_negativ = games[games.recommended==False].sample(frac=0.8, random_state=random_state)
train_games = pd.concat([train_games_positiv, train_games_negativ])
test_games = games.drop(train_games.index)

#Set Traindata for music
train_music_positiv = music[music.rating.isin(['8', '9', '10', '11', '12', '13', '14', '15'])].sample(frac=0.8, random_state=random_state)
train_music_negativ = music[music.rating.isin(['1', '2', '3', '4', '5', '6', '7'])].sample(frac=0.8, random_state=random_state)
train_music = pd.concat([train_music_positiv, train_music_negativ])
test_music = music.drop(train_music.index)

#Set Traindata for apps
train_apps_positiv = apps[apps.Rating >= 3].sample(frac=0.8, random_state=random_state)
train_apps_negativ = apps[apps.Rating < 3].sample(frac=0.8, random_state=random_state)
train_apps = pd.concat([train_apps_positiv, train_apps_negativ])
test_apps = apps.drop(train_apps.index)

# Spalten vereinheitlichen

In [106]:
#Einfügen positiv Spalte und Text
#games.reset_index(inplace=True)
train_games['Positiv'] = (games.recommended)
train_games['Text'] = (games.content_text)
train_games[:10]

,content_text,recommended,Positiv,Text
27954,Der absolut beste Teil der Serie! Hab zuvor Fa...,True,True,Der absolut beste Teil der Serie! Hab zuvor Fa...
25306,Von den Trailern und Reviews bekommt man ja gu...,True,True,Von den Trailern und Reviews bekommt man ja gu...
12535,"Als ich von theHunter: Primal gehört habe, hat...",True,True,"Als ich von theHunter: Primal gehört habe, hat..."
14045,Yay!,True,True,Yay!
6696,ETS 2 ein einfach geniales Spiel ;),True,True,ETS 2 ein einfach geniales Spiel ;)
13857,Meiner Meinung nach eines der besten und logis...,True,True,Meiner Meinung nach eines der besten und logis...
20836,"Es gibt zwar noch den ein oder anderen Bug*, a...",True,True,"Es gibt zwar noch den ein oder anderen Bug*, a..."
16646,"Man sollte zuerst sagen, dass das Spiel nichts...",True,True,"Man sollte zuerst sagen, dass das Spiel nichts..."
10932,sehr gutes Spiel feier ich persöhnlich auch im...,True,True,sehr gutes Spiel feier ich persöhnlich auch im...
2378,http://youtu.be/ty0Ty1CiQ5U,True,True,http://youtu.be/ty0Ty1CiQ5U


In [108]:
#Einfügen positiv Spalte und Text
#games.reset_index(inplace=True)
test_games['Positiv'] = (games.recommended)
test_games['Text'] = (games.content_text)
test_games[:10]

,content_text,recommended,Positiv,Text
3,leider ist die grafic für den absoluter mist u...,False,False,leider ist die grafic für den absoluter mist u...
12,nice the game!,True,True,nice the game!
13,Super Spiel,True,True,Super Spiel
18,Leider ein Fehlkauf ;-(,False,False,Leider ein Fehlkauf ;-(
23,Das is scheiße!!!! Holt es euch nicht!,False,False,Das is scheiße!!!! Holt es euch nicht!
27,Also eines der besten Sonic Spiele die ich jem...,True,True,Also eines der besten Sonic Spiele die ich jem...
39,"Die beste Bananenrepublik, die man für Geld ka...",True,True,"Die beste Bananenrepublik, die man für Geld ka..."
44,"Cooles Game, zumindest was die 1.te knappe Stu...",True,True,"Cooles Game, zumindest was die 1.te knappe Stu..."
50,Wir haben es jetzt einige Stunden im lokalen 2...,True,True,Wir haben es jetzt einige Stunden im lokalen 2...
51,da es kostenlos ist lohnt sich das spiel auf j...,True,True,da es kostenlos ist lohnt sich das spiel auf j...


In [109]:
#Einfügen positiv Spalte und Text
train_apps = apps[apps.Rating!=3]
train_apps.reset_index(inplace=True)
train_apps['Positiv'] = (apps.Rating > 3)
train_apps[:10]

C:\Users\IBM_ADMIN\Anaconda3\Ana\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,index,Heading,Rating,Text,Positiv
0,0,Widgets?,4,"App ist hervorragend designt, mir fehlt aber d...",True
1,1,Super,5,Passt,True
2,2,Klasse Wecker,4,"Er ist recht einfach zu bedienen, selbst im Ha...",True
3,3,None,5,Echt super.nur zum ausstellen des Weckers muss...,True
4,4,Echt cool,5,Das ist der erste wecker den man hört :D also ...,True
5,6,Sehr guter app,5,Welle gelek rind e. Morovan ji xewê radike.,False
6,7,Klasse,5,Klasse,True
7,8,Sehr gut,5,"Wecker, Countdown, Stoppuhr als Erweiterung. M...",True
8,9,None,5,"Testversion verspricht viel, soweit funzt alle...",True
9,10,Bitte um Fehlerbehebung,2,"Ich nutze diese APP seid Jahren ,war immer sup...",True


In [110]:
#Einfügen positiv Spalte und Text
test_apps = apps[apps.Rating!=3]
test_apps.reset_index(inplace=True)
test_apps['Positiv'] = (apps.Rating > 3)
test_apps[:10]

C:\Users\IBM_ADMIN\Anaconda3\Ana\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,index,Heading,Rating,Text,Positiv
0,0,Widgets?,4,"App ist hervorragend designt, mir fehlt aber d...",True
1,1,Super,5,Passt,True
2,2,Klasse Wecker,4,"Er ist recht einfach zu bedienen, selbst im Ha...",True
3,3,None,5,Echt super.nur zum ausstellen des Weckers muss...,True
4,4,Echt cool,5,Das ist der erste wecker den man hört :D also ...,True
5,6,Sehr guter app,5,Welle gelek rind e. Morovan ji xewê radike.,False
6,7,Klasse,5,Klasse,True
7,8,Sehr gut,5,"Wecker, Countdown, Stoppuhr als Erweiterung. M...",True
8,9,None,5,"Testversion verspricht viel, soweit funzt alle...",True
9,10,Bitte um Fehlerbehebung,2,"Ich nutze diese APP seid Jahren ,war immer sup...",True


In [28]:
#Einfügen positiv Spalte und Text
#music_positiv = music[music.rating.isin(['8', '9', '10', '11', '12', '13', '14', '15'])]
#music_negativ = music[music.rating.isin(['1', '2', '3', '4', '5', '6', '7'])]
train_music['Text'] = (music.description + music.summary)
#train_music.reset_index(inplace=True)
train_music['Positiv'] = (music.rating.isin(['8', '9', '10', '11', '12', '13', '14', '15']))
train_music

,description,rating,summary,Text,Positiv
11964,"Sechs Songs in rund 22 Minuten, davon einer do...",8,"FAZIT: Wenn ich gewusst hätte, das mir über...","Sechs Songs in rund 22 Minuten, davon einer do...",True
4766,"Es ist unbestreitbar, dass s Stammband UNDERW...",11,FAZIT: Hinsichtlich experimenteller Sounds kan...,"Es ist unbestreitbar, dass s Stammband UNDERW...",True
1663,"Russland, das Land der „lupenreinen Demokraten...",14,FAZIT: Wenn Schönheit und musikalische Eleganz...,"Russland, das Land der „lupenreinen Demokraten...",True
2954,""" "" ist optisch nicht mehr ganz so pompös wie ...",12,FAZIT: Der metallisch-progressiv-alternative M...,""" "" ist optisch nicht mehr ganz so pompös wie ...",True
5458,Eine traurige Nachricht überschattet die Veröf...,9,FAZIT: Das B-Movie unter den Prog-Metal-Epigon...,Eine traurige Nachricht überschattet die Veröf...,True
8003,In den ersten Sekunden rumpelt 80er-Jahre-Hard...,14,FAZIT: In der Anwartschaft auf den Titel des b...,In den ersten Sekunden rumpelt 80er-Jahre-Hard...,True
1985,"Es gibt Leute, für die ist Brian Johnson auch ...",13,FAZIT: Die Hardrock-Scheibe des Jahres? Bis je...,"Es gibt Leute, für die ist Brian Johnson auch ...",True
1219,"Karisma, norwegische Band, was sonst? Neu ist,...",13,FAZIT: Hört jetzt rein und wisst später über d...,"Karisma, norwegische Band, was sonst? Neu ist,...",True
4901,2012 warteten SUSPYRE aus New Jersey in Eigeni...,11,"FAZIT: Für SUSPYRE gilt aktuell, ihre Bemühung...",2012 warteten SUSPYRE aus New Jersey in Eigeni...,True
7965,Das letzte Album von erschien vor über zehn ...,12,"FAZIT: Elf Punkte gibt es für die Musik, einen...",Das letzte Album von erschien vor über zehn ...,True


In [29]:
test_music['Text'] = (music.description + music.summary)
#test_music.reset_index(inplace=True)
test_music['Positiv'] = (music.rating.isin(['8', '9', '10', '11', '12', '13', '14', '15']))
test_music

,description,rating,summary,Text,Positiv
1,"Der Sticker auf dieser offeriert uns sofort,...",-,FAZIT: Der holländische Hardcore-Fünfer SUPERH...,"Der Sticker auf dieser offeriert uns sofort,...",False
3,Ein abschreckender Titel! Abschreckend für all...,-,FAZIT: Das Doppelalbum „Ballett 1 & 2“ von e...,Ein abschreckender Titel! Abschreckend für all...,False
4,„Der Instrumental-König entfesselt einen Monsu...,12,FAZIT: So sollte klassischer instrumentaler Pr...,„Der Instrumental-König entfesselt einen Monsu...,True
8,Drei Jahre Zeit steckten in dieses erste Alb...,11,FAZIT: Überdurchschnittlich gute Platte im zei...,Drei Jahre Zeit steckten in dieses erste Alb...,True
9,Als Garant für Roots Rock von hoher Güte gilt ...,10,FAZIT: s aktuelles Album ist ein durchweg tra...,Als Garant für Roots Rock von hoher Güte gilt ...,True
14,Was vom Einerlei der Folkrock und -Metal-Sch...,8,FAZIT: nehmen ihre Musik ernster als sich s...,Was vom Einerlei der Folkrock und -Metal-Sch...,True
18,PROJECT ARMAGEDDON aus dem texanischen Houston...,-,FAZIT: Traditioneller Doom Metal mit nicht bew...,PROJECT ARMAGEDDON aus dem texanischen Houston...,False
22,Mit ihrer näselnden Stimme und aufgrund der Ar...,7,FAZIT: Aus der irischen Provinz und dennoch ve...,Mit ihrer näselnden Stimme und aufgrund der Ar...,False
25,Konzertdarbietungen fing bislang stets im in...,-,"FAZIT: Mit "" "" bringt - 62 Jahre jung - nich...",Konzertdarbietungen fing bislang stets im in...,False
26,Nach der beeindruckenden Entwicklung zum Garan...,-,"FAZIT: Ehrlich, in dieser Form sind unschlag...",Nach der beeindruckenden Entwicklung zum Garan...,False


# GAMES
Vectorizing

In [113]:
#Vectorizer Games
vectorizer = CountVectorizer(min_df=10, max_df=200, token_pattern='\w+')
vectorizer.fit(train_games.Text)

X = vectorizer.transform(train_games.Text)
y = train_games.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_games.Positiv
predicted = clf.predict(vectorizer.transform(test_games.Text))
accuracy_score(actuals, predicted)

0.75987306064880111

GAMES: Minimal Werte geändert

In [119]:
vectorizer = CountVectorizer(min_df=25, max_df=200, token_pattern='\w+')
vectorizer.fit(train_games.Text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=200, max_features=None, min_df=25,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='\\w+', tokenizer=None,
        vocabulary=None)

In [120]:
X = vectorizer.transform(train_games.Text)
y = train_games.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_games.Positiv
predicted = clf.predict(vectorizer.transform(test_games.Text))
accuracy_score(actuals, predicted)

0.7455923836389281

In [121]:
vectorizer = CountVectorizer(min_df=50, max_df=200, token_pattern='\w+')
vectorizer.fit(train_games.Text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=200, max_features=None, min_df=50,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='\\w+', tokenizer=None,
        vocabulary=None)

In [122]:
X = vectorizer.transform(train_games.Text)
y = train_games.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_games.Positiv
predicted = clf.predict(vectorizer.transform(test_games.Text))
accuracy_score(actuals, predicted)

0.72320169252468269

In [123]:
vectorizer = CountVectorizer(min_df=100, max_df=200, token_pattern='\w+')
vectorizer.fit(train_games.Text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=200, max_features=None, min_df=100,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='\\w+', tokenizer=None,
        vocabulary=None)

In [124]:
X = vectorizer.transform(train_games.Text)
y = train_games.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_games.Positiv
predicted = clf.predict(vectorizer.transform(test_games.Text))
accuracy_score(actuals, predicted)

0.69146685472496472

GAMES: Maximal Werte geändert

In [125]:
vectorizer = CountVectorizer(min_df=10, max_df=100, token_pattern='\w+')
vectorizer.fit(train_games.Text)

X = vectorizer.transform(train_games.Text)
y = train_games.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_games.Positiv
predicted = clf.predict(vectorizer.transform(test_games.Text))
accuracy_score(actuals, predicted)

0.73219322990126945

In [135]:
vectorizer = CountVectorizer(min_df=10, max_df=300, token_pattern='\w+')
vectorizer.fit(train_games.Text)

X = vectorizer.transform(train_games.Text)
y = train_games.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_games.Positiv
predicted = clf.predict(vectorizer.transform(test_games.Text))
accuracy_score(actuals, predicted)

0.77362482369534558

In [136]:
vectorizer = CountVectorizer(min_df=10, max_df=400, token_pattern='\w+')
vectorizer.fit(train_games.Text)

X = vectorizer.transform(train_games.Text)
y = train_games.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_games.Positiv
predicted = clf.predict(vectorizer.transform(test_games.Text))
accuracy_score(actuals, predicted)

0.78385049365303239

GAMES: Zwei Varianten Min- und Max-Anzahl

In [128]:
vectorizer = CountVectorizer(min_df=25, max_df=25, token_pattern='\w+')
vectorizer.fit(train_games.Text)

X = vectorizer.transform(train_games.Text)
y = train_games.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_games.Positiv
predicted = clf.predict(vectorizer.transform(test_games.Text))
accuracy_score(actuals, predicted)

0.58921015514809594

In [129]:
vectorizer = CountVectorizer(min_df=25, max_df=300, token_pattern='\w+')
vectorizer.fit(train_games.Text)

X = vectorizer.transform(train_games.Text)
y = train_games.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_games.Positiv
predicted = clf.predict(vectorizer.transform(test_games.Text))
accuracy_score(actuals, predicted)

0.7625176304654443

GAMES: Nur Adjektive

GAMES: Nur Adjektive, Adverbien, Nomen und Verben

GAMES: Negation

# Apps

Vectorizing

APPS: Minimalwerte geändert

In [131]:
vectorizer = CountVectorizer(min_df=105, max_df=200, token_pattern='\w+')
vectorizer.fit(train_apps.Text)

X = vectorizer.transform(train_apps.Text)
y = train_apps.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_apps.Positiv
predicted = clf.predict(vectorizer.transform(test_apps.Text))
accuracy_score(actuals, predicted)

0.79519799319118434

In [137]:
vectorizer = CountVectorizer(min_df=25, max_df=200, token_pattern='\w+')
vectorizer.fit(train_apps.Text)

X = vectorizer.transform(train_apps.Text)
y = train_apps.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_apps.Positiv
predicted = clf.predict(vectorizer.transform(test_apps.Text))
accuracy_score(actuals, predicted)

0.79639252224810364

In [132]:
vectorizer = CountVectorizer(min_df=50, max_df=200, token_pattern='\w+')
vectorizer.fit(train_apps.Text)

X = vectorizer.transform(train_apps.Text)
y = train_apps.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_apps.Positiv
predicted = clf.predict(vectorizer.transform(test_apps.Text))
accuracy_score(actuals, predicted)

0.7954368990025682

In [133]:
vectorizer = CountVectorizer(min_df=100, max_df=200, token_pattern='\w+')
vectorizer.fit(train_apps.Text)

X = vectorizer.transform(train_apps.Text)
y = train_apps.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_apps.Positiv
predicted = clf.predict(vectorizer.transform(test_apps.Text))
accuracy_score(actuals, predicted)

0.79513826673833843

APPS: Maximal Werte geändert

In [134]:
vectorizer = CountVectorizer(min_df=10, max_df=100, token_pattern='\w+')
vectorizer.fit(train_apps.Text)

X = vectorizer.transform(train_apps.Text)
y = train_apps.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_apps.Positiv
predicted = clf.predict(vectorizer.transform(test_apps.Text))
accuracy_score(actuals, predicted)

0.80081227975870517

In [138]:
vectorizer = CountVectorizer(min_df=10, max_df=300, token_pattern='\w+')
vectorizer.fit(train_apps.Text)

X = vectorizer.transform(train_apps.Text)
y = train_apps.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_apps.Positiv
predicted = clf.predict(vectorizer.transform(test_apps.Text))
accuracy_score(actuals, predicted)

0.80105118557008903

In [139]:
vectorizer = CountVectorizer(min_df=10, max_df=400, token_pattern='\w+')
vectorizer.fit(train_apps.Text)

X = vectorizer.transform(train_apps.Text)
y = train_apps.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_apps.Positiv
predicted = clf.predict(vectorizer.transform(test_apps.Text))
accuracy_score(actuals, predicted)

0.80039419458878336

APPS: Zwei Varianten Min- und Max-Anzahl

In [140]:
vectorizer = CountVectorizer(min_df=25, max_df=25, token_pattern='\w+')
vectorizer.fit(train_apps.Text)

X = vectorizer.transform(train_apps.Text)
y = train_apps.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_apps.Positiv
predicted = clf.predict(vectorizer.transform(test_apps.Text))
accuracy_score(actuals, predicted)

0.79513826673833843

In [141]:
vectorizer = CountVectorizer(min_df=25, max_df=300, token_pattern='\w+')
vectorizer.fit(train_apps.Text)

X = vectorizer.transform(train_apps.Text)
y = train_apps.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_apps.Positiv
predicted = clf.predict(vectorizer.transform(test_apps.Text))
accuracy_score(actuals, predicted)

0.79591471062533592

APPS: Nur Adjektive

APPS: Nur Adjektive, Adverbien, Nomen und Verben

APPS: Negation

# Music
Vaectorizing

In [30]:
vectorizer = CountVectorizer(min_df=10, max_df=200, token_pattern='\w+')
vectorizer.fit(train_music.Text)

X = vectorizer.transform(train_music.Text)
y = train_music.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_music.Positiv
predicted = clf.predict(vectorizer.transform(test_music.Text))
accuracy_score(actuals, predicted)

0.49523396880415943

MUSIC: Minimal Werte geändert

In [24]:
vectorizer = CountVectorizer(min_df=25, max_df=200, token_pattern='\w+')
vectorizer.fit(train_music.Text)

X = vectorizer.transform(train_music.Text)
y = train_music.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_music.Positiv
predicted = clf.predict(vectorizer.transform(test_music.Text))
accuracy_score(actuals, predicted)

0.62651646447140386

In [25]:
vectorizer = CountVectorizer(min_df=50, max_df=200, token_pattern='\w+')
vectorizer.fit(train_music.Text)

X = vectorizer.transform(train_music.Text)
y = train_music.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_music.Positiv
predicted = clf.predict(vectorizer.transform(test_music.Text))
accuracy_score(actuals, predicted)

0.64059792027729634

In [145]:
vectorizer = CountVectorizer(min_df=100, max_df=200, token_pattern='\w+')
vectorizer.fit(train_music.Text)

X = vectorizer.transform(train_music.Text)
y = train_music.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_music.Positiv
predicted = clf.predict(vectorizer.transform(test_music.Text))
accuracy_score(actuals, predicted)

0.67222703639514736

MUSIC: Maximal Werte geänder

In [146]:
vectorizer = CountVectorizer(min_df=10, max_df=100, token_pattern='\w+')
vectorizer.fit(train_music.Text)

X = vectorizer.transform(train_music.Text)
y = train_music.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_music.Positiv
predicted = clf.predict(vectorizer.transform(test_music.Text))
accuracy_score(actuals, predicted)

0.63648180242634311

In [147]:
vectorizer = CountVectorizer(min_df=10, max_df=300, token_pattern='\w+')
vectorizer.fit(train_music.Text)

X = vectorizer.transform(train_music.Text)
y = train_music.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_music.Positiv
predicted = clf.predict(vectorizer.transform(test_music.Text))
accuracy_score(actuals, predicted)

0.64081455805892551

In [148]:
vectorizer = CountVectorizer(min_df=10, max_df=400, token_pattern='\w+')
vectorizer.fit(train_music.Text)

X = vectorizer.transform(train_music.Text)
y = train_music.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_music.Positiv
predicted = clf.predict(vectorizer.transform(test_music.Text))
accuracy_score(actuals, predicted)

0.64103119584055457

MUSIC: Zwei Varianten Min- und Max-Anzahl

In [149]:
vectorizer = CountVectorizer(min_df=25, max_df=25, token_pattern='\w+')
vectorizer.fit(train_music.Text)

X = vectorizer.transform(train_music.Text)
y = train_music.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_music.Positiv
predicted = clf.predict(vectorizer.transform(test_music.Text))
accuracy_score(actuals, predicted)

0.72183708838821492

In [151]:
vectorizer = CountVectorizer(min_df=25, max_df=300, token_pattern='\w+')
vectorizer.fit(train_music.Text)

X = vectorizer.transform(train_music.Text)
y = train_music.Positiv
clf = MultinomialNB().fit(X, y)
actuals = test_music.Positiv
predicted = clf.predict(vectorizer.transform(test_music.Text))
accuracy_score(actuals, predicted)

0.62889948006932406

 MUSIC: Nur Adjektive

MUSIC: Nur Adjektive, Adverbien, Nomen und Verben

MUSIC: Negation